In [1]:
import pyspark as ps
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType
from pyspark.sql.functions import mean, max, min, count
import pymysql
#1.1: Load/Read the CompanyABC Stock(CompanyABC stock.csv) data into SparkSQL Dataframe [ hint read() ].
spark = SparkSession.builder.appName('CompanyApp').getOrCreate()
companydf = spark.read.load("C:/Users/trigg/Downloads/CompanyABC_stock.csv", format="csv", header = True,inferSchema = True)
companydf.columns
companydf.printSchema()
companydf.describe().show()
companydf.show()




root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.38999

In [4]:
#1.2: Print out the first five rows.
companydf.head(5)

[Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

In [6]:
#1.3: Create a new Dataframe column called “HV Ratio,” which will stimulate the ratio of the High price versus the total Volume of stock that was traded for a day. 
companydf2 = companydf.withColumn('HV Ratio', companydf['High']/companydf['Volume'])
companydf2.select('HV Ratio').show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



In [13]:
#1.4: Find out on what day the stock price was the highest. (Hint: use the High column.)

companydf.orderBy(companydf.High.desc()).head(1)[0][0]


datetime.datetime(2015, 1, 13, 0, 0)

In [17]:
#1.5: What is the average (mean) closing price? (Hint: Use the Close column.)
companydf.select(mean('Close')).show()


+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [19]:
#1.6: What is the maximum and minimum volume of stock traded? (Hint: Use the Volume column).

companydf.select(max("Volume"),min("Volume")).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



In [21]:
#1.7: For how many days was the closing value less than 70 dollars? (Hint: Use the count() method.)

companydf.filter(companydf['Close'] < 70).count()


397

In [46]:
#1.8: What percentage of the time was the High greater than 80 dollars?
companydf.filter('High > 80').count() * 100/companydf.count()

9.141494435612083

In [4]:
#1.9: Create a database named CompanyABC_DB using MariaDB(HeidiSQL).  
Host = 'localhost'
User ='root'
Password='Trey2002' 
conn = pymysql.connect(host=Host,user=User,password=Password)
cur  = conn.cursor()
cur.execute("CREATE DATABASE CompanyABC_DB") 
cur.execute("SHOW DATABASES")
databaseList = cur.fetchall()
for database in databaseList:
    print(database)
conn.close()

ProgrammingError: (1007, "Can't create database 'companyabc_db'; database exists")

In [ ]:
#1.10: Load/Write CompanyABC stock.csv file data into CompanyABC_DB database  from SparkSQL Dataframe. You can specify any table name for that file.
companydf.write.format('jdbc').mode('append').option('url','jdbc:mysql://localhost:3306/CompanyABC_DB').option('dbtable','CompanyABC_DB.companyabc_stock').option('user','root').option('password','Trey2002').save()


: 

In [21]:
#2.1: Load/Read both CompanyABC sales datasets (Sales_April_2019.csv and Sales_February_2019.csv) into SparkSQL Dataframe ( hint read() ).


product= spark.read.load(['C:/Users/trigg/Downloads/Sales_April_2019.csv','C:/Users/trigg/Downloads/Sales_February_2019.csv'], format='csv',header=True,inferSchema= True)
product.columns
product.printSchema()
product.describe().show()
product.show()
product.columns
product.printSchema()
product.describe().show()
product.show()

root
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)

+-------+------------------+------------+-------------------+------------------+--------------+--------------------+
|summary|          Order ID|     Product|   Quantity Ordered|        Price Each|    Order Date|    Purchase Address|
+-------+------------------+------------+-------------------+------------------+--------------+--------------------+
|  count|             30275|       30328|              30275|             30275|         30328|               30328|
|   mean|173816.63355904212|        null| 1.1239966969446737| 183.7621383980421|          null|                null|
| stddev|14902.179833784816|        null|0.43431295924749175|328.79347229907665|          null|                null|
|    min|            150502|20i

In [24]:
#2.2 If you use the above command on the sales dataset, you will notice that each Order has “Price Each” and “Quantity Ordered” columns, but the “Total price” is missing. See below:
#Now, create a new Dataframe column called “Total price” and find the “Total price” of the Order for both sales files as shown in the screenshot below:



productdf= product.withColumn('Total Price',product['Price Each'] * product['Quantity Ordered'])
productdf.show()



+--------+--------------------+----------------+----------+---------------+--------------------+-----------+
|Order ID|             Product|Quantity Ordered|Price Each|     Order Date|    Purchase Address|Total Price|
+--------+--------------------+----------------+----------+---------------+--------------------+-----------+
|  176558|USB-C Charging Cable|               2|     11.95| 4/19/2019 8:46|917 1st St, Dalla...|       23.9|
|  176559|Bose SoundSport H...|               1|     99.99| 4/7/2019 22:30|682 Chestnut St, ...|      99.99|
|  176560|        Google Phone|               1|     600.0|4/12/2019 14:38|669 Spruce St, Lo...|      600.0|
|  176560|    Wired Headphones|               1|     11.99|4/12/2019 14:38|669 Spruce St, Lo...|      11.99|
|  176561|    Wired Headphones|               1|     11.99| 4/30/2019 9:27|333 8th St, Los A...|      11.99|
|  176562|USB-C Charging Cable|               1|     11.95|4/29/2019 13:03|381 Wilson St, Sa...|      11.95|
|  176563|Bose Soun

In [25]:
#2.3: Load/Write sales data from SparkSQL Dataframe into CompanyABC_DB database. You can specify any n:ame to the table. Remember “Total price” must be recorded. Your result will look similar /closer to the screenshot below
productdf.write.format('jdbc').mode('append').option('url','jdbc:mysql://localhost:3306/CompanyABC_DB').option('dbtable','CompanyABC_DB.TotalPrice').option('user','root').option('password','Trey2002').save()